## IV. Methodology

### 4.1 Dataset Preparation
* ~Load the clean EEG, EOG, and EMG data from the EEGDenoiseNet dataset in .npy format.~
### 4.2 Contaminated Signal Generation
For each selected clean EEG segment:
* ~Randomly select an artifact segment from each chosen type.~
* ~Calculate the contaminated signal using the formula: contaminated_signal = clean_eeg + λ * artifact_segment.~
* ~Randomly generate and use λ value. Then calculate the SNR of the generated contaminated signal.~
* ~With the new array of contaminated signals, drop those that are out of the typical SNR range:~
    * Ocular artifacts: [-7 to 2]
    * Mygoenic artifact: [-7 to 4]

### 4.3 Denoising Methods
#### 4.3.1 Empirical Mode Decomposition
* Apply EMD to the contaminated EEG signal to decompose it into intrinsic mode functions (IMFs).
* Analyze the IMFs to identify and remove artifacts or noise components.
* Reconstruct the denoised EEG signal using the remaining IMFs.
* Measure the processing time and SNR of the denoised signal.

#### 4.3.2 Independent Component Analysis
* Apply ICA to the contaminated EEG signal to extract independent components.
* Identify the components representing the EEG signal.
* Reconstruct the denoised EEG signal using the selected components.
* Measure the processing time and SNR of the denoised signal.

### 4.4 Evaluation
* Calculate the SNR of the denoised EEG signals using appropriate metrics such as the ratio of signal power to noise power.
* Compare the SNR performance and processing time of ICA and EMD for denoising the EEG signals.
* Analyze the results and draw conclusions regarding the denoising effectiveness and efficiency of each method.

### 4.1 Dataset Preparation

In [ ]:
import random
import numpy as np
from PyEMD import EMD
import matplotlib.pyplot as plt
import memory_profiler
import time
from memory_profiler import memory_usage

In [ ]:
eeg = np.load('EEGDenoiseNet/EEG_all_epochs.npy')
print(f"------EEG-----\nSize: {eeg.size}\nRow count: {len(eeg)}\nColumn count: {len(eeg[0])}\nFirst Column: {eeg[:, 0]}\n\n")

emg = np.load('EEGDenoiseNet/EMG_all_epochs.npy')
print(f"------EMG(Heart)-----\nSize: {emg.size}\nRow count: {len(emg)}\nColumn count: {len(emg[0])}\nFirst Column: {emg[:, 0]}\n\n")

eog = np.load('EEGDenoiseNet/EOG_all_epochs.npy')
print(f"------EOG(Ocular)-----\nSize: {eog.size}\nRow count: {len(eog)}\nColumn count: {len(eog[0])}\nFirst Column: {eog[:, 0]}")

### 4.2 Contaminated Signal Generation

* Randomly select an artifact segment from each chosen type.
* Calculate the contaminated signal using the formula: contaminated_signal = clean_eeg + λ * artifact_segment.
* Randomly generate and use λ value. Then calculate the SNR of the generated contaminated signal.

In [ ]:
def get_random_rows(dataset, num_rows):
    num_total_rows = dataset.shape[0]
    selected_indices = np.random.choice(num_total_rows, size=num_rows, replace=False)
    selected_rows = dataset[selected_indices, :]
    return selected_rows


def calculate_snr(clean_eeg, artifact_segment,λ):
    N = 512
    # Calculate the signal power (clean EEG)
    signal_power = (np.sum(clean_eeg ** 2)*(1/N))**0.5
    # Calculate the noise power (artifact segment)
    noise_power = (np.sum((λ*artifact_segment)**2)*(1/N))**0.5
    # Calculate the SNR in dB (RMS FORMULA)
    snr_db = 10 * np.log10(signal_power / noise_power)
    return round(snr_db,2)

def generate_contaminated_signal(clean_eeg_data,artifact_data,num_samples:int, artifact_type:str):
    num_clean_eeg_samples = clean_eeg_data.shape[0]
    num_eog_artifacts = artifact_data.shape[0]

    contaminated_eeg_data = []
    contamination_indices = []
    snr_values=[]

    for i in range(num_samples):
        clean_eeg_index = np.random.randint(0, num_clean_eeg_samples)
        eog_artifact_index = np.random.randint(0, num_eog_artifacts)

        clean_eeg_sample = clean_eeg_data[clean_eeg_index]
        artifact = artifact_data[eog_artifact_index]

        #GENERATE RANDOM LAMBDA VALUE HERE
        λ = random.uniform(-40,40)
        #GENERATE CONTAMINATED SIGNAL USING FORMULA: y=x+(λ*n)
        contaminated_eeg_sample = clean_eeg_sample + (artifact*λ)
        snr=calculate_snr(clean_eeg_data,artifact,λ)

        contaminated_eeg_data.append(contaminated_eeg_sample)
        contamination_indices.append((clean_eeg_index, eog_artifact_index))
        snr_values.append(snr)
    return np.array(contaminated_eeg_data), contamination_indices, snr_values, artifact_type

In [ ]:
contaminated_eeg_data,indices,snr_values,artifact_type = generate_contaminated_signal(eeg,eog,100000,'ocular')
print(f"{contaminated_eeg_data}\nSNR Values: {snr_values}\nArtifact Type: {artifact_type}")

KeyboardInterrupt: 

* With the new array of contaminated signals, drop those that are out of the typical SNR range:
    * Ocular artifacts: [-7 to 2]
    * Mygoenic artifact: [-7 to 4]


**Specific Steps**
* ~Find the data type of `contaminated_eeg_data`, `indices`, `snr_values`, `artifact_type`~
    * **array**: `contaminated_eeg_data`
    * **list**: `indices`, `snr_values`
    * **tuple**: elements inside `indices`
    * **string**: `artifact_type`
* ~Go through `snr_values`. If it goes out the range of the given value, find the index of that value.~
 * ~Use that index to drop values from `contaminated_eeg_data`, `indices`, `snr_values`, and `artifact_type`~
* ~Find the length of the new array and take n samples from this. This will be your sample size.~

In [ ]:
count, out_of_range = 0,len(contaminated_eeg_data)
contaminated_data2,indices2,snr_values2 = [],[],[]
if artifact_type =='ocular':
    snr_range = [-7,2]
elif artifact_type == 'myogenic':
    snr_range = [-7,4]

for i in snr_values:
    if i>=snr_range[0] and i<=snr_range[1]:
        index=snr_values.index(i)
        count+=1
        out_of_range-=1
        contaminated_data2.append(contaminated_eeg_data[index])
        indices2.append(indices[index])
        snr_values2.append(snr_values[index])
    if len(contaminated_data2)==2000:
        break

print(f"Original size: {len(contaminated_eeg_data)}\nNumber of out of range SNR values: {out_of_range}\nRemaining sample size: {count}")

print(f'\nSAMPLE SIZE OR New length of contaminated_eeg_data: {len(contaminated_data2)}')

### 4.3 Denoising Algorithms

#### 4.3.1 Empirical Mode Decomposition

* Apply EMD to the contaminated EEG signal to decompose it into intrinsic mode functions (IMFs) and a residual or trend.
* Analyze the IMFs to identify and remove artifacts or noise components.
* Reconstruct the denoised EEG signal using the remaining IMFs.
* Measure the processing time and SNR of the denoised signal.

**NOTES**
* `IMF` is a numpy array that contains `n` numbers of `imf`s.
* `imf` is a numpy array that is a subset of imf that contains the actual signal inputs.
* In the following example, `IMF` has 7 elements or `imf`s. Each `imf` file contains 512 rows.

**SPECIFIC STEPS**
* Create an empty list to put the IMFs to be generated.
* Create a function that goes through every row of `contaminated_data2`.
    * Denoise the given `contaminated_data2` row into IMFs.
    * Use: ```n, imf = emurate(IMF)```

In [ ]:
def apply_emd(s):
    sampling_rate = 256  # Hz
    duration = 2  # seconds
    t = np.arange(0, duration, 1/sampling_rate)

    emd = EMD(DTYPE=np.float16, max_imfs=2)

    # Start memory & time usage recording
    start_time = time.time()
    mem_usage_start = memory_usage()[0]

    #EMD
    IMF = EMD().emd(s,t)

    # End memory & time usage recording
    mem_usage_end = memory_usage()[0]
    end_time = time.time()

    #Return memory & time usage results
    mem_usage = mem_usage_end-mem_usage_start
    execution_time = end_time - start_time

    return IMF, mem_usage, execution_time

In [ ]:
IMF_data, memory_data, time_data = [],[],[]
for sample in contaminated_data2:
    IMF_temp, mem_temp, time_temp = apply_emd(sample)
    IMF_data.append(IMF_temp)
    memory_data.append(mem_temp)
    time_data.append(time_temp)

#### 4.3.2 Independent Component Analysis

#### 4.4 Evaluation